# Example of using Wave2Vec2 for sentiment analysis and embedding

In [11]:
%%capture
%pip install numpy
%pip install torch
%pip install transformers
%pip install soundfile
%pip install kagglehub
%pip install librosa

In [12]:
import numpy as np
import torch
import torch.nn as nn
from transformers import Wav2Vec2Processor
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2Model,
    Wav2Vec2PreTrainedModel,
)

In [13]:
class RegressionHead(nn.Module):
    r"""Classification head."""

    def __init__(self, config):

        super().__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):

        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)

        return x

In [14]:
class EmotionModel(Wav2Vec2PreTrainedModel):
    r"""Speech emotion classifier."""

    def __init__(self, config):

        super().__init__(config)

        self.config = config
        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = RegressionHead(config)
        self.init_weights()

    def forward(
            self,
            input_values,
    ):

        outputs = self.wav2vec2(input_values)
        hidden_states = outputs[0]
        hidden_states = torch.mean(hidden_states, dim=1)
        logits = self.classifier(hidden_states)

        return hidden_states, logits

In [15]:
import soundfile as sf
import os, random
import kagglehub
import librosa
from IPython.display import Audio

In [16]:
# load model from hub
device = 'mps'
model_name = 'audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim'
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = EmotionModel.from_pretrained(model_name).to(device)

# load files from hub
folder_path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio") + '/Actor_01' #from 01 to 24

print("Path to dataset files:", folder_path)

Path to dataset files: /Users/ronan/.cache/kagglehub/datasets/uwrfkaggler/ravdess-emotional-speech-audio/versions/1/Actor_01


In [17]:

# # dummy signal
# sampling_rate = 16000
# read the FLAC file
# pick a random .wav file from the folder
file_path = os.path.join(folder_path, random.choice([f for f in os.listdir(folder_path) if f.endswith('.wav')]))

signal, sampling_rate = sf.read(file_path)
print(file_path, sampling_rate)
# Resample the signal to 16000 Hz
if sampling_rate != 16000:
    print('resampling to 16000 Hz')
    signal = librosa.resample(signal, orig_sr=sampling_rate, target_sr=16000)
    sampling_rate = 16000
# signal = np.zeros((1, sampling_rate), dtype=np.float32)

Audio(signal, rate=sampling_rate)

/Users/ronan/.cache/kagglehub/datasets/uwrfkaggler/ravdess-emotional-speech-audio/versions/1/Actor_01/03-01-01-01-02-01-01.wav 48000
resampling to 16000 Hz


In [18]:
def process_func(
    x: np.ndarray,
    sampling_rate: int,
    embeddings: bool = False,
) -> np.ndarray:
    r"""Predict emotions or extract embeddings from raw audio signal."""

    # run through processor to normalize signal
    # always returns a batch, so we just get the first entry
    # then we put it on the device
    y = processor(x, sampling_rate=sampling_rate)
    y = y['input_values'][0]
    y = y.reshape(1, -1)
    y = torch.from_numpy(y).to(device)

    # run through model
    with torch.no_grad():
        y = model(y)[0 if embeddings else 1]

    # convert to numpy
    y = y.detach().cpu().numpy()

    return y

In [24]:
print('Arousal    dominance valence')
print(process_func(signal, sampling_rate))

print(process_func(signal, sampling_rate, embeddings=True))
# Pooled hidden states of last transformer layer
# [[-0.00752167  0.0065819  -0.00746342 ...  0.00663632  0.00848748
#    0.00599211]]

Arousal    dominance valence
[[0.39127    0.49816188 0.3452627 ]]
[[-0.00759058  0.00593888 -0.00711144 ...  0.00711803  0.00850544
   0.00522046]]
